<a href="https://colab.research.google.com/github/darkraithromb/next-platform-starter/blob/main/uav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gymnasium[all]
!pip install stable-baselines3[extra]
!pip install --upgrade stable-baselines3
import shimmy
import gym
from gym import spaces
import numpy as np
from stable_baselines3 import PPO

class UAVHandoverEnv(gym.Env):
    def __init__(self):
        super(UAVHandoverEnv, self).__init__()


        self.action_space = spaces.Discrete(2)

        self.observation_space = spaces.Box(low=np.array([-np.inf, -np.inf, -np.inf, 0, 0, -np.inf]),
                                            high=np.array([np.inf, np.inf, np.inf, 10, 45, 0]),
                                            dtype=np.float32)

        self.state = None


        self.base_stations = self.initialize_base_stations()

    def initialize_base_stations(self):

        bs_positions = []
        for i in range(45):
            bs_positions.append([np.random.uniform(0, 6), np.random.uniform(0, 6), 0.3])
        return np.array(bs_positions)

    def reset(self):

        self.state = np.array([0, 0, 0, np.random.uniform(1, 3), 0, -70])
        return self.state

    def compute_rssi(self, position, base_station_id):

        bs_position = self.base_stations[base_station_id]
        distance = np.linalg.norm(position - bs_position)
        rssi = -20 * np.log10(distance + 1) - 70
        return rssi

    def step(self, action):
        x, y, z, velocity, current_bs_id, rssi = self.state


        if action == 1:
            new_bs_id = np.random.randint(0, len(self.base_stations))
            rssi = self.compute_rssi(np.array([x, y, z]), new_bs_id)
            penalty = -1
        else:

            rssi = self.compute_rssi(np.array([x, y, z]), int(current_bs_id))
            new_bs_id = current_bs_id
            penalty = 0


        x += np.random.uniform(-0.1, 0.1) * velocity
        y += np.random.uniform(-0.1, 0.1) * velocity

        x = np.clip(x, 0, 6)
        y = np.clip(y, 0, 6)


        reward = rssi / -100
        reward += penalty


        self.state = np.array([x, y, z, velocity, new_bs_id, rssi])


        done = rssi < -90

        return self.state, reward, done, {}

    def render(self, mode='human'):

        pass


env = UAVHandoverEnv()


model = PPO("MlpPolicy", env, verbose=1)

model.learn(total_timesteps=10000)

model.save("ppo_uav_handover")


obs = env.reset()
for _ in range(1000):
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    if done:
        obs = env.reset()
    env.render()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 11.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error origina

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 1039 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 716         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.020455036 |
|    clip_fraction        | 0.53        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.674      |
|    explained_variance   | -0.0239     |
|    learning_rate        | 0.0003      |
|    loss                 | 1.03        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0604     |
|    value_loss           | 5.09        |
-----------------------------------------
----------------------------------